In [ ]:
import pandas as pd
import os
import seaborn as sns

In [ ]:
result_files = os.listdir("benchmarking")
all_results = []

#for rfile in result_files:
#    df = pd.read_csv(os.path.join("benchmarking", rfile))
#    ds = rfile[:rfile.index("0.2")-1]
#    df["dataset"] = ds
#    all_results.append(df)

#discard the first warmup run
for rfile in result_files:
    df = pd.read_csv(os.path.join("benchmarking", rfile))
    ds = rfile[:rfile.index("0.2")-1]
    df["dataset"] = ds
    all_results.append(df.loc[1:])

res_df = pd.concat(all_results, ignore_index=True)
res_df["efficiency"] = res_df["efficiency"].apply(lambda x: float(x[len("tensor("):-1]))

In [ ]:
res_df.groupby(["dataset", "method"])["efficiency"].mean()

In [ ]:
# exclude weird extra value by taking the middle three
#res_df = res_df.groupby(["dataset", "method"])["runtime"].apply(lambda x: x.drop([x.idxmax(), x.idxmin()])).reset_index()

#def group_result(x):
#    x_mean = x.mean()
#    x_max = x.max()
#    x_min = x.min()
#    x_dev = max(x_mean - x_min, x_max - x_mean)
#    return f"{x_mean:.2f} $\pm$ {x_dev:.2f}"
#results_table = res_df.groupby(["dataset", "method"])["runtime"].apply(group_result).reset_index().pivot(index="dataset", columns="method", values="runtime")

def group_results(x):
    x_mean = x.mean()
    x_std = x.std()
    return f"{x_mean:.2f} $\pm$ {1.96 * x_std:.2f}"

results_table = res_df.groupby(["dataset", "method"])["runtime"].apply(group_results).reset_index().pivot(index="dataset", columns="method", values="runtime")

In [ ]:
print(results_table.reindex(["CiteSeer", "Amazon_Photos", "Cora", "PubMed", "Coauthor_CS", "Coauthor_Physics"]).to_latex())

In [ ]:
res_df.groupby(["dataset", "method"])["runtime"].mean().reset_index().groupby("dataset").apply(lambda x: x[x.method=="baseline"]["runtime"].item()/x[x.method=="cache+batch"]["runtime"].item())